In [11]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from bokeh.io import output_notebook, show  # TODO change output
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [115]:
# Import a gridgraph
from gridgraph.dynamic_grid import Element, Grid

# Quick unit test
crit_radius = 1
coords = [(0,0), (0,1), (0,2),
          (1,0), (1,1), (1,2),
          (2,0), (2,1), (2,2),]
mygrid = Grid(coords=coords,
              element_class=Element,
              crit_radius=crit_radius)

for e in mygrid.elements:
    e.target = e.neighbors[0]

walk = mygrid.walk_graph(mygrid.elements[0])
print(walk)  # Here is a bottom-up traversal starting at element 0
print('check: 0, 3, 6, 1, 4, 7, 2, 5, 8')

deque([0, 3, 6, 1, 4, 7, 2, 5, 8])
check: 0, 3, 6, 1, 4, 7, 2, 5, 8


In [148]:
# Display in Bokeh
import math

from bokeh.models import GraphRenderer, Circle, StaticLayoutProvider, Plot, Range1d
from bokeh.palettes import Spectral9
from bokeh.plotting import figure

#plot = figure(title='Graph Visualizer', x_range=(-1, 3), y_range=(-1, 3),
#              tools='', toolbar_location=None)
plot = Plot(x_range=Range1d(-.2, 2.2), y_range=Range1d(-.2, 2.2))
plot.background_fill_color = "midnightblue"
plot.background_fill_alpha = .15


node_indices = list(range(len(mygrid)))

# Mesh renderer
mesh = GraphRenderer()

N=9

edge_dict = {}
edge_dict['start'], edge_dict['end'] = mygrid.mesh()
# edge_dict['property'] = mygrid.ws   # or something
mesh.edge_renderer.data_source.data = edge_dict
mesh.edge_renderer.glyph.line_width = 1
mesh.edge_renderer.glyph.line_dash = [2, 4]
mesh.edge_renderer.glyph.line_color = 'silver'


# Graph renderer
graph = GraphRenderer()

edge_dict = {}
edge_dict['start'], edge_dict['end'] = mygrid.edges()
graph.edge_renderer.data_source.data = edge_dict
graph.edge_renderer.glyph.line_width = 2
graph.edge_renderer.glyph.line_color = 'red'
# TODO colormap based on current, width based on wire width


# Node renderer
nodes = GraphRenderer()
nodes.node_renderer.data_source.add(node_indices, 'index')
nodes.node_renderer.data_source.add(Spectral9, 'color')
nodes.node_renderer.glyph = Circle(size=7, fill_color='lime')
# TODO special markers for the sinks


### start of layout code
layout = mygrid.layout()
mesh.layout_provider = StaticLayoutProvider(graph_layout=layout)
graph.layout_provider = StaticLayoutProvider(graph_layout=layout)
nodes.layout_provider = StaticLayoutProvider(graph_layout=layout)

plot.renderers.append(mesh)
plot.renderers.append(graph)
plot.renderers.append(nodes)

show(plot)

In [ ]:
# Export and render in HTML, then host the thing.